In [8]:
import numpy as np 
import tensorflow as tf
from matplotlib import pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data
import random
import cv2
import scipy.misc
import os
from PIL import Image

In [9]:
#读取mnist数据集
def read_mnist_data(file_path):
    mnist_data = input_data.read_data_sets(file_path,one_hot=True)
    train_data = mnist_data.train.images
    train_labels = mnist_data.train.labels
    test_data = mnist_data.test.images
    test_labels = mnist_data.test.labels
    valid_data,valid_labels = mnist_data.validation.images,mnist_data.validation.labels
    return train_data,train_labels,test_data,test_labels,valid_data,valid_labels

In [10]:
#合并训练集和检验集
def combination_train_valid(train_data,train_labels,valid_data,valid_labels):
    train_data = np.vstack((train_data,valid_data))
    train_labels = np.vstack((train_labels,valid_labels))
    return train_data,train_labels

In [11]:
#查看mnist数据集中的手写数字
def image_show(image_data):
    temp_image = image_data.reshape(28,28)
    plt.figure(figsize=(1,1))
    plt.imshow(temp_image,cmap='gray')
    plt.show()

In [12]:
#保存mnist数据
def save_mnist_data(image,file_path,number,data_type,label):
    #mnist = input_data.read_data_sets('./MNIST_data/',one_hot=True)
    if os.path.exists(file_path) is False:
        os.makedirs(file_path)
    temp_image = image.reshape(28,28)
    temp_img_file = file_path + data_type + '_mnist_data_%d_label_%d.jpg'%(number+1,label)
    Image.fromarray((temp_image*255).astype('uint8'),mode='L').convert('RGB').save(temp_img_file)

In [13]:
#保存所有图片
def save_images(file_path_1,file_path_2,train_data,train_labels,test_data,test_labels):
    train_length = len(train_data)
    for i in range(train_length):
        save_mnist_data(train_data[i],file_path_1,i,'train',int(np.where(train_labels[i]==1)[0]))
    test_length = len(test_data)
    for j in range(test_length):
        save_mnist_data(test_data[j],file_path_2,j,'test',int(np.where(test_labels[j]==1)[0]))

In [14]:
#数据变换函数
def mnist_transform_function(image,resize_type,filepath,number,label):
    #先构建64*64的背景图片
    temp_fig = np.zeros([64,64])
    plt.imshow(temp_fig,cmap='gray')
    #plt.imsave('temp.png',temp_fig,cmap='gray')
    '''
    cv2.imshow('background',temp_fig)
    cv2.imwrite('background.png',temp_fig)
    '''
    #将28*28的图像转为8*8
    temp_img = image.reshape(28,28)
    temp_img_resize = cv2.resize(temp_img,(resize_type,resize_type),interpolation=cv2.INTER_NEAREST)
    #plt.imshow(temp_img,cmap='gray')
    #plt.imshow(temp_img_resize,cmap='gray')
    #将resize后的图片随机放入background的某一个地方
    positions = 64 / resize_type
    height = random.randint(0,positions-1) * resize_type
    width = random.randint(0,positions-1) * resize_type
    #print(height,width)
    temp_fig[height:height+resize_type,width:width+resize_type] = temp_img_resize
    #plt.imshow(temp_fig,cmap='gray')
    temp_fig_path = filepath+'MNIST_train_%d_label_%d.png'%(number,label)
    plt.imsave(temp_fig_path,temp_fig,cmap='gray')

In [15]:
#处理所有的数据
def local_task_data(train_data,train_labels,resize_type,filepath):
    train_length = len(train_data)
    for i in range(train_length):
        mnist_transform_function(train_data[i],resize_type,filepath,i,int(np.where(train_labels[i]==1)[0]))      

In [16]:
'''
if __name__ == "__main__":
    file_path = './MNIST_data/'
    train_data,train_labels,test_data,test_labels,valid_data,valid_labels = read_mnist_data(file_path)
    #print(train_data.shape,train_labels.shape,test_data.shape,test_labels.shape,valid_data.shape,valid_labels.shape)
    train_data,train_labels = combination_train_valid(train_data,train_labels,valid_data,valid_labels)
    print(train_data.shape,train_labels.shape)
    image_file_path = './MNIST_IMG/'
    train_file_path = './MNIST_IMG/train/'
    test_file_path = './MNIST_IMG/test/'
    #save_images(train_file_path,test_file_path,train_data,train_labels,test_data,test_labels)
    #mnist_transform_function(train_data[2],8)
    transform_train_data_filepath = './MNIST_global_task/train/'
    transform_test_data_filepath = './MNIST_global_task/test/'
    local_task_data(test_data,test_labels,32,transform_test_data_filepath)
'''   

'\nif __name__ == "__main__":\n    file_path = \'./MNIST_data/\'\n    train_data,train_labels,test_data,test_labels,valid_data,valid_labels = read_mnist_data(file_path)\n    #print(train_data.shape,train_labels.shape,test_data.shape,test_labels.shape,valid_data.shape,valid_labels.shape)\n    train_data,train_labels = combination_train_valid(train_data,train_labels,valid_data,valid_labels)\n    print(train_data.shape,train_labels.shape)\n    image_file_path = \'./MNIST_IMG/\'\n    train_file_path = \'./MNIST_IMG/train/\'\n    test_file_path = \'./MNIST_IMG/test/\'\n    #save_images(train_file_path,test_file_path,train_data,train_labels,test_data,test_labels)\n    #mnist_transform_function(train_data[2],8)\n    transform_train_data_filepath = \'./MNIST_global_task/train/\'\n    transform_test_data_filepath = \'./MNIST_global_task/test/\'\n    local_task_data(test_data,test_labels,32,transform_test_data_filepath)\n'

In [17]:
from keras.models import Model,load_model
from keras.layers import Dense,Dropout,Lambda,Flatten,Conv2D,MaxPool2D,Input,AveragePooling2D
from keras.optimizers import Adam,RMSprop
from keras.utils.np_utils import to_categorical
import keras.backend as K
from keras.engine.topology import Layer
from keras.datasets import mnist

Using TensorFlow backend.


In [18]:
'''
读取文件夹下的所有数据
'''
def read_transfor_data(filepath):
    labels = []
    img_data = []
    for filename in os.listdir(r"./"+filepath):
        labels.append(int(filename[-5]))
        img = cv2.imread(filepath + "/" + filename)
        img_data.append(img)
    #print(img_data[0].shape)
    #print(labels)  

In [19]:
#数据变换函数
def img_transform_function(image,resize_type):
    #先构建64*64的背景图片
    temp_fig = np.zeros([64,64])
    #plt.imshow(temp_fig,cmap='gray')
    #plt.imsave('temp.png',temp_fig,cmap='gray')
    '''
    cv2.imshow('background',temp_fig)
    cv2.imwrite('background.png',temp_fig)
    '''
    #将28*28的图像转为8*8
    temp_img = image.reshape(28,28)
    temp_img_resize = cv2.resize(temp_img,(resize_type,resize_type),interpolation=cv2.INTER_NEAREST)
    #plt.imshow(temp_img,cmap='gray')
    #plt.imshow(temp_img_resize,cmap='gray')
    #将resize后的图片随机放入background的某一个地方
    positions = 64 / resize_type
    height = random.randint(0,positions-1) * resize_type
    width = random.randint(0,positions-1) * resize_type
    #print(height,width)
    temp_fig[height:height+resize_type,width:width+resize_type] = temp_img_resize
    #plt.imshow(temp_fig,cmap='gray')
    #temp_fig_path = filepath+'MNIST_train_%d_label_%d.png'%(number,label)
    #plt.imsave(temp_fig_path,temp_fig,cmap='gray')
    plt.imshow(temp_fig,cmap='gray')
    return temp_fig

In [20]:
#处理所有的图片
def handle_all_images(x_train,resize_type):
    x_train_transformed = []
    for i in x_train:
        temp_img = img_transform_function(i,resize_type)
        x_train_transformed.append(temp_img)
    x_train_transformed = np.array(x_train_transformed)
    return x_train_transformed

In [21]:
optimizer = RMSprop(lr=0.001,rho=0.9,epsilon=1e-08,decay=0.0)

Instructions for updating:
Colocations handled automatically by placer.


In [31]:
def ConvAC_Model_Construction_Wide_base(r):
    input_img = Input(shape=(64,64,1))
    #representation层（使用3*3的卷积）
    X1 = Conv2D(10,(3,3),activation='relu',padding='same')(input_img)
    
    X1 = Conv2D(4*r,(1,1),activation='relu')(X1)
    X1 = MaxPool2D(pool_size=(2,2),strides=2)(X1)
    X1 = Dropout(0.25)(X1)
    
    X1 = Conv2D(4*r,(1,1),activation='relu')(X1)
    X1 = MaxPool2D(pool_size=(2,2),strides=2)(X1)
    X1 = Dropout(0.25)(X1)
    
    X1 = Conv2D(2*r,(1,1),activation='relu')(X1)
    X1 = MaxPool2D(pool_size=(2,2),strides=2)(X1)
    X1 = Dropout(0.25)(X1)

    X1 = Conv2D(2*r,(1,1),activation='relu')(X1)
    X1 = MaxPool2D(pool_size=(2,2),strides=2)(X1)
    X1 = Dropout(0.25)(X1)

    X1 = Conv2D(r,(1,1),activation='relu')(X1)
    X1 = MaxPool2D(pool_size=(2,2),strides=2)(X1)
    X1 = Dropout(0.25)(X1)

    X1 = Conv2D(r,(1,1),activation='relu')(X1)
    X1 = MaxPool2D(pool_size=(2,2),strides=2)(X1)
    X1 = Dropout(0.25)(X1)
    
    X1 = Flatten()(X1)
    X1 = Dense(10,activation="relu")(X1)
    X1 = Dropout(0.5)(X1)
    ConvAC_output = Dense(10,activation = "softmax")(X1)
    
    ConvAC_model = Model(input_img,ConvAC_output)
    return ConvAC_model

In [32]:
def ConvAC_Model_Construction_Wide_tip(r):
    input_img = Input(shape=(64,64,1))
    #representation层（使用3*3的卷积）
    X1 = Conv2D(10,(3,3),activation='relu',padding='same')(input_img)
    
    X1 = Conv2D(r,(1,1),activation='relu')(X1)
    X1 = MaxPool2D(pool_size=(2,2),strides=2)(X1)
    X1 = Dropout(0.25)(X1)
    
    X1 = Conv2D(r,(1,1),activation='relu')(X1)
    X1 = MaxPool2D(pool_size=(2,2),strides=2)(X1)
    X1 = Dropout(0.25)(X1)
    
    X1 = Conv2D(2*r,(1,1),activation='relu')(X1)
    X1 = MaxPool2D(pool_size=(2,2),strides=2)(X1)
    X1 = Dropout(0.25)(X1)

    X1 = Conv2D(2*r,(1,1),activation='relu')(X1)
    X1 = MaxPool2D(pool_size=(2,2),strides=2)(X1)
    X1 = Dropout(0.25)(X1)

    X1 = Conv2D(4*r,(1,1),activation='relu')(X1)
    X1 = MaxPool2D(pool_size=(2,2),strides=2)(X1)
    X1 = Dropout(0.25)(X1)

    X1 = Conv2D(4*r,(1,1),activation='relu')(X1)
    X1 = MaxPool2D(pool_size=(2,2),strides=2)(X1)
    X1 = Dropout(0.25)(X1)
    
    X1 = Flatten()(X1)
    X1 = Dense(10,activation="relu")(X1)
    X1 = Dropout(0.5)(X1)
    ConvAC_output = Dense(10,activation = "softmax")(X1)
    
    ConvAC_model = Model(input_img,ConvAC_output)
    return ConvAC_model

In [33]:
def ConvAC_model_fit_wide_base(X_train,y_train,X_test,y_test,epochs,batch_size,r,optimizer):
    ConvAC_model = ConvAC_Model_Construction_Wide_tip(r)
    ConvAC_model.compile(optimizer = optimizer,
                         loss = "categorical_crossentropy",
                         metrics = ["accuracy"])
    hist_temp = ConvAC_model.fit(X_train,y_train,
                                 batch_size=batch_size,
                                 epochs=epochs,
                                 verbose = 1,
                                 )
    train_accuracy = hist_temp.history['acc'][-1]
    test_result = ConvAC_model.evaluate(x=X_test,y=y_test)
    test_accuracy = test_result[1]
    return train_accuracy,test_accuracy

In [34]:
def ConvAC_model_fit_wide_tip(X_train,y_train,X_test,y_test,epochs,batch_size,r,optimizer):
    ConvAC_model = ConvAC_Model_Construction_Wide_base(r)
    ConvAC_model.compile(optimizer = optimizer,
                         loss = "categorical_crossentropy",
                         metrics = ["accuracy"])
    hist_temp = ConvAC_model.fit(X_train,y_train,
                                 batch_size=batch_size,
                                 epochs=epochs,
                                 verbose = 1,
                                 )
    train_accuracy = hist_temp.history['acc']
    test_result = ConvAC_model.evaluate(x=X_test,y=y_test)
    test_accuracy = test_result[1]
    return train_accuracy,test_accuracy

In [35]:
if __name__ == "__main__":
    (X_train,y_train),(X_test,y_test) = mnist.load_data()
    assert K.image_data_format() == "channels_last"
    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)
#     resize_type_1 = 8
#     resize_type_2 = 32
#     a = img_transform_function(X_train[0],resize_type_1)
#     b = img_transform_function(X_train[1],resize_type_2)
#     x_transformed_train_local = handle_all_images(X_train,resize_type_1)
#     x_transformed_train_local = np.reshape(x_transformed_train_local,(-1,64,64,1))/255
#     x_transformed_test_local = handle_all_images(X_test,resize_type_1)
#     x_transformed_test_local = np.reshape(x_transformed_test_local,(-1,64,64,1))/255
#     x_transformed_train_global = handle_all_images(X_train,resize_type_2)
#     x_transformed_train_global = np.reshape(x_transformed_train_global,(-1,64,64))/255
#     x_transformed_test_global = handle_all_images(X_test,resize_type_2)
#     x_transformed_test_global = np.reshape(x_transformed_test_global,(-1,64,64))/255
#     np.save('local_task_mnist_train',x_transformed_train_local)
#     np.save('local_task_mnist_test',x_transformed_test_local)
#     np.save('global_task_mnist_train',x_transformed_train_global)
#     np.save('global_task_mnist_test',x_transformed_test_global)
    global_task_train = np.load('global_task_mnist_train.npy')
    global_task_test = np.load('global_task_mnist_test.npy')
    print("global task 数据加载完毕！")
    epochs = 3
    batch_size = 100
    wide_base_train_accuracy = []
    wide_base_test_accuracy = []
    wide_tip_train_accuracy = []
    wide_tip_test_accuracy = []
    for r in range(5,31,5):
        base_train_acc,base_test_acc = ConvAC_model_fit_wide_base(global_task_train,y_train,global_task_test,y_test,epochs,batch_size,r,optimizer)
        wide_base_train_accuracy.append(base_train_acc)
        wide_base_test_accuracy.append(base_test_acc)
        tip_train_acc,tip_test_acc = ConvAC_model_fit_wide_tip(global_task_train,y_train,global_task_test,y_test,epochs,batch_size,r,optimizer)
        wide_tip_train_accuracy.append(tip_train_acc)
        wide_tip_test_accuracy.append(tip_test_acc)
        print("global task %d round is finished."%(r))
    np.save('wide_base_train_accuracy',np.array(wide_base_train_accuracy))
    np.save('wide_base_test_accuracy',np.array(wide_base_test_accuracy))
    np.save('wide_tip_train_accuracy',np.array(wide_tip_train_accuracy))
    np.save('wide_tip_test_accuracy',np.array(wide_tip_test_accuracy))
    #local task result
    local_task_train = np.load("local_task_mnist_train.npy")
    local_task_test = np.load("local_task_mnist_test.npy")
    print("local task 数据加载完毕！")
    local_wide_base_train_accuracy = []
    local_wide_base_test_accuracy = []
    local_wide_tip_train_accuracy = []
    local_wide_tip_test_accuracy = []
    for k in range(5,31,5):
        local_base_train_acc,local_base_test_acc = ConvAC_model_fit_wide_base(local_task_train,y_train,local_task_test,y_test,epochs,batch_size,k,optimizer)
        local_wide_base_train_accuracy.append(local_base_train_acc)
        local_wide_base_test_accuracy.append(local_base_test_acc)
        local_tip_train_acc,local_tip_test_acc = ConvAC_model_fit_wide_tip(local_task_train,y_train,local_task_test,y_test,epochs,batch_size,k,optimizer)
        local_wide_tip_train_accuracy.append(local_tip_train_acc)
        local_wide_tip_test_accuracy.append(local_tip_test_acc)   
        print("local task %d round is finished."%(k))
    np.save('local_wide_base_train_accuracy',np.array(local_wide_base_train_accuracy))
    np.save('local_wide_base_test_accuracy',np.array(local_wide_base_test_accuracy))
    np.save('local_wide_tip_train_accuracy',np.array(local_wide_tip_train_accuracy))
    np.save('local_wide_tip_test_accuracy',np.array(local_wide_tip_test_accuracy))

ValueError: Negative dimension size caused by subtracting 2 from 1 for 'max_pooling2d_155/MaxPool' (op: 'MaxPool') with input shapes: [?,1,1,40].

NameError: name 'wide_base_train_accuracy' is not defined

5
10
15
20
25
30


In [4]:
a

range(6, 31, 5)